In [6]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [7]:
!python -V

Python 3.10.12


In [20]:
import pickle
import pandas as pd
import numpy as np
import os

In [9]:
with open('./hw4/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [10]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [11]:
df = read_data('./data/yellow_tripdata_2023-03.parquet')

In [12]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [14]:
sd = np.std(y_pred)

print("Standard Deviation of predictions:", sd)

Standard Deviation of predictions: 6.247488852238703


In [23]:
year = 2023  # Example year
month = 3    # Example month
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

# Create a DataFrame with the predictions
df_result = pd.DataFrame({
    'ride_id': df['ride_id'],
    'predictions': y_pred
})

# Define the output file path
output_file = './output/results.parquet'

# Save the DataFrame as a Parquet file
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

# Check the size of the output file
file_size = os.path.getsize(output_file)
print(f"Size of the output file: {file_size} bytes")

Size of the output file: 68641783 bytes
